In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.initializers import GlorotUniform, HeUniform

In [2]:
train_datagen = ImageDataGenerator(
    rescale=1./255
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '../data/cropped/',
    target_size=(300, 300),
    batch_size=32,
    class_mode="categorical")

validation_generator = test_datagen.flow_from_directory(
    '../data/cropped/',
    target_size=(300, 300),
    batch_size=32,
    class_mode="categorical")

Found 5850 images belonging to 3 classes.
Found 5850 images belonging to 3 classes.


In [54]:
cnn_first = keras.Sequential([
    keras.layers.Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(300, 300, 3), kernel_initializer=HeUniform(seed=1)),
    keras.layers.MaxPooling2D((2, 2), strides=2),
    keras.layers.Conv2D(64, (3,3), padding='same', activation='relu', kernel_initializer=HeUniform(seed=1)),
    keras.layers.MaxPooling2D((2, 2), strides=2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu', kernel_initializer=HeUniform(seed=1)),
    keras.layers.Dense(3,  activation='softmax')
])
cnn_first.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 300, 300, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 150, 150, 32)     0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 150, 150, 64)      18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 75, 75, 64)       0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 360000)            0         
                                                                 
 dense_6 (Dense)             (None, 128)              

In [58]:
my_callbacks = [
    keras.callbacks.EarlyStopping(patience=5),
    keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
]

In [59]:
cnn_first.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [60]:
first = cnn_first.fit_generator(train_generator, 
                       epochs=5,
                       verbose=1,
                       validation_data=validation_generator,
                       validation_freq=1,
                       class_weight=None,
                       initial_epoch=0)

/tmp/ipykernel_17829/3411962650.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  cnn_first.fit_generator(train_generator,


Epoch 1/5
183/183 [==============================] - 117s 636ms/step - loss: 0.2890 - accuracy: 0.9279 - val_loss: 0.3058 - val_accuracy: 0.9032
Epoch 2/5
183/183 [==============================] - 118s 647ms/step - loss: 0.1626 - accuracy: 0.9508 - val_loss: 0.0854 - val_accuracy: 0.9738
Epoch 3/5
183/183 [==============================] - 118s 643ms/step - loss: 0.0949 - accuracy: 0.9723 - val_loss: 0.0633 - val_accuracy: 0.9824
Epoch 4/5
183/183 [==============================] - 118s 642ms/step - loss: 0.0901 - accuracy: 0.9786 - val_loss: 0.0641 - val_accuracy: 0.9814
Epoch 5/5
183/183 [==============================] - 116s 634ms/step - loss: 0.0816 - accuracy: 0.9762 - val_loss: 0.0365 - val_accuracy: 0.9909


In [71]:
cnn_first.save('simple_nn_from_documentation')

INFO:tensorflow:Assets written to: simple_nn_from_documentation/assets


INFO:tensorflow:Assets written to: simple_nn_from_documentation/assets


In [14]:
from keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input, decode_predictions

In [4]:
model = VGG19(weights='imagenet')

574710816/574710816 [==============================] - 121s 0us/step


In [19]:
img = image.load_img("gfx100s_sample_04_thum-1.jpg", target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

In [20]:
decode_predictions(model.predict(x))

1/1 [==============================] - 1s 633ms/step


[[('n09193705', 'alp', 0.8820469),
  ('n03958227', 'plastic_bag', 0.036847375),
  ('n09468604', 'valley', 0.015771553),
  ('n03792972', 'mountain_tent', 0.005355929),
  ('n09246464', 'cliff', 0.0051444555)]]

In [22]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.inception_v3 import InceptionV3

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(128, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(3, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# train the model on the new data for a few epochs
model.fit(train_generator, 
          epochs=5,
          verbose=1,
          validation_data=validation_generator,
          validation_freq=1,
          class_weight=None,
          initial_epoch=0)

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit(...)

87910968/87910968 [==============================] - 17s 0us/step
Epoch 1/5
 20/183 [==>...........................] - ETA: 3:50 - loss: 0.2756

KeyboardInterrupt: 